In [1]:
# EJG Animal Shelter Dashboard Application - Enhanced Version
# Author: Edward Garcia
# Overview:
# This script represents an enhanced version of my Animal Shelter Dashboard, originally developed for my CS340 course.
# My enhancements aim to improve user experience, functionality, and maintainability of the dashboard through UI/UX improvements, additional interactive
# features, and logging. I also added in a structured header 

# Enhancements in this Revised Version:
# 1. User Interface and User Experience:
#    - Integrated Dash Bootstrap components to provide better styling and responsiveness.
#    - Applied the VAPOR Bootstrap theme to create a visually appealing and consistent interface.
#    - Improved data presentation by utilizing cards, colors, custom fonts, and shadow effects to headers for a more engaging user interface.

# 2. Navigation and Search Functionality:
#    - Added a responsive navigation bar with links to the About Us and Contact Us sections, enabling easy navigation that scrolls to the page section.
#    - Introduced a search bar in the navigation to filter animal data by breed or name, with modifications to ensure search queries are 
#      case-insensitive for a more user-friendly experience.

# 3. Data Management Enhancements:
#    - Added a "Refresh Data" button, allowing users to easily update and view the most current dataset.
#    - Added a "Download Data" button to provide users with the ability to download the animal dataset as a CSV file directly from the dashboard.

# 4. User Engagement Features:
#    - Included dedicated sections for "About Us" and "Contact Us" to improve user engagement and provide relevant organizational information.

# 5. Logging and Maintainability:
#    - Implemented logging to capture and track dashboard activities, which aids in debugging, application monitoring, and maintaining behavior.

# Setup the Jupyter version of Dash
from dash import Dash
import dash_bootstrap_components as dbc  # Added Dash Bootstrap components for enhanced UI/UX.
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import dash_leaflet as dl
import plotly.express as px
import base64
import pandas as pd
import io
import logging # Here I imported the logging componenent for strucutured loggin of the application. 

# Here I added logging to track events, debugging, and monitor application behavior.
logging.basicConfig(
    filename='animal_shelter_dashboard.log',  # File to save logs related to the dashboard
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# Import the AnimalShelter class for CRUD operations
from animal_shelter_CRUD_revised import AnimalShelter

###############################
# Data Manipulation / Model
###############################

# This is the MongoDB username and password I setup for the docker container running MongoDB
username = "edwardgarcia5_snhu"
password = "password"

# Establish a connection to the database using the AnimalShelter class
shelter = AnimalShelter(username, password, 'host.docker.internal', 27017, 'AAC', 'animals')

# Retrieve all records from the database
df = pd.DataFrame.from_records(shelter.read({}))

# Drop the '_id' column only if it exists
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

###############################
# Dashboard Layout / View
###############################

# This is where I added the Dash Bootstrap theme called VAPOR for a more modern and engaging look.
app = Dash(__name__, external_stylesheets=[dbc.themes.VAPOR])

# Here I loaded and encoded the Grazioso Salvare logo to display it in the navigation bar.
image_filename = 'Grazioso Salvare Logo.png'  
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Here I defined the dashboard layout with new navigation and filter options for enhanced interactivity.
app.layout = dbc.Container([
    # Here I added a navigation bar with branding and a search feature.
    dbc.Navbar(
        dbc.Container([
            dbc.NavbarBrand(
                "EJG's Animal Shelter",
                className="me-2",
                style={
                    'color': '#f0e31a',  # Here I enhanced branding with a bright color for better visibility.
                    'fontWeight': 'bold',
                    'fontSize': '2rem',
                    'textShadow': '2px 2px 4px rgba(0, 0, 0, 0.5)',  # Here I added a shadow effect to the branding text.
                    'letterSpacing': '1.3px'
                }
            ),
            # Here I added navigation links for About Us and Contact Us sections to improve user engagement.
            dbc.Nav(
                [
                    dbc.NavItem(dbc.NavLink("About Us", href="#about-us", external_link=True)),
                    dbc.NavItem(dbc.NavLink("Contact Us", href="#contact", external_link=True)),
                ],
                className="ms-auto"
            ),
            # Here I added a search input directly into the navigation bar to allow easy filtering by breed or name.
            dcc.Input(id='search-input', type='text', placeholder='Search by breed or name...', debounce=True, style={'width': '300px'}),
        ]),
        color="primary", dark=True, className="mb-4"
    ),

    # Logo Section with Central Placement
    dbc.Row([
        dbc.Col(
            html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height': '10%', 'width': '10%'})),
            width=12
        )
    ], className="mb-4"),

    # Filter and Data Table Section
    dbc.Row([
        # Here I implemented enhanced filter options for better usability and user interaction.
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H4("Filter Options", id="filter-options"), style={'color': '#000000'}),
                dbc.CardBody([
                    dcc.RadioItems(
                        id='filter-type',
                        options=[
                            {'label': 'Water Rescue', 'value': 'Water Rescue'},
                            {'label': 'Mountain Rescue', 'value': 'Mountain or Wilderness Rescue'},
                            {'label': 'Disaster Rescue', 'value': 'Disaster or Individual Tracking'},
                            {'label': 'Reset', 'value': 'Reset'}
                        ],
                        value='Reset',
                        labelStyle={'display': 'block', 'margin-bottom': '10px', 'color': '#000000'},
                        style={'font-size': '18px'},
                    ),
                    # Here I added a "Refresh Data" button to allow users to update the table if new data is added.
                    dbc.Button("Refresh Data", id="refresh-button", color="primary", className="mt-3 me-2"),
                    
                    # Here I added a "Download Data" button to allow users to download the current dataset as a CSV file.
                    dbc.Button("Download Data", id="download-button", color="info", className="mt-3")
                ])
            ], style={'backgroundColor': '#e7f9c3'})
        ], width=3),

        # The Data Table Section has been visually optimized by altering parameters and editing styles. 
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H4("Animal Data", id="animal-data"), style={'color': '#000000'}),
                dbc.CardBody([
                    dash_table.DataTable(
                        id='datatable-id',
                        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                        data=df.to_dict('records'),
                        editable=True,
                        row_selectable="single",
                        filter_action="native",
                        sort_action="native",
                        page_action="native",
                        page_current=0,
                        page_size=20,
                        style_table={'overflowX': 'auto', 'height': '400px', 'minWidth': '100%'},
                        style_cell={
                            'textAlign': 'left',
                            'padding': '10px',
                            'fontFamily': 'Arial, sans-serif',  # Here I added a modern font style
                            'fontSize': '14px',
                            'backgroundColor': '#ffffff',
                            'color': '#000000',
                        },
                        style_header={
                            'backgroundColor': '#343a40',
                            'color': '#ffffff',
                            'fontWeight': 'bold',
                            'border': '1px solid #333'
                        },
                        style_data_conditional=[
                            {'if': {'row_index': 'odd'}, 'backgroundColor': '#f9f9f9'},
                            {'if': {'row_index': 'even'}, 'backgroundColor': '#ffffff'},
                            {'if': {'state': 'selected'}, 'backgroundColor': '#D2F3FF', 'border': '1px solid #0074D9'}  
                        ]
                    )
                ])
            ], style={'backgroundColor': '#f7df34'})  # Here I added a background color to the data card to make it more visually engaging.
        ], width=9)
    ], className="mb-4"),

    # Here I added the Download component to facilitate CSV download functionality.
    dcc.Download(id="download-dataframe-csv"),

    # Div for Charts and Map
    # Here I added a pie chart for breed distribution to provide the user with visual data insights.
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H4("Breed Distribution Chart", id="breed-chart"), style={'color': '#000000'}),
                dbc.CardBody([
                    dcc.Graph(id='graph-id')  
                ])
            ], style={'backgroundColor': '#fae5e5'})
        ], width=6),

        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H4("Animal Location Map", id="animal-map"), style={'color': '#000000'}),
                dbc.CardBody([
                    dl.Map(id='map-id', style={'width': '100%', 'height': '400px'}, zoom=10, center=(30.75, -97.48), children=[dl.TileLayer()])
                ])
            ], style={'backgroundColor': '#e5f6fa'})
        ], width=6)
    ]),

    # Here I updated the About Us section to better represent the Animal Shelter organization and engage with users.
dbc.Row([
    dbc.Col([
        html.Div([
            html.H3("About Us", id="about-us", style={'color': '#ffffff'}),
            html.P(
                """
                At EJG's Animal Shelter, our mission is to support and enhance animal rescues by providing a powerful, user-friendly dashboard application. 
                The Animal Shelter Dashboard was created to help rescue centers, animal caregivers, and adoption facilitators manage and analyze rescue 
                animal data more effectively. We have crafted this platform to simplify data tracking, make animal information more accessible, and ensure 
                a smoother rescue and adoption process. Our solution allows users to visualize data, filter records based on specific criteria such as 
                breed or rescue type, and even pinpoint animal locations on an interactive map.
                
                By integrating these features, we empower our partners—whether rescue teams, volunteers, or potential adopters—to make informed decisions 
                and work more efficiently. This dashboard, developed by Edward Garcia, leverages modern tools and technologies to ensure data is accurate, 
                secure, and easy to interact with. We continuously work on improving the application to create a meaningful difference in the lives of 
                rescued animals, and provide a transparent, organized view for all involved in the animal adoption journey.
                
                Our ultimate goal is to promote the welfare of animals by leveraging technology to enhance collaboration, efficiency, and, ultimately, 
                the rate of successful adoptions. Thank you for being part of our journey to help animals find loving homes.
                """
            )
        ], style={'backgroundColor': '#a013eb', 'padding': '20px', 'borderRadius': '10px'})
    ])
], className="mt-5 mb-5"),


    # Here I added the Contact Us section for optimized communication and user queries.
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H3("Contact Us", id="contact", style={'color': '#ffffff'}),
                html.P("For more information, please contact Edward Garcia at edward@example.com.")
            ], style={'backgroundColor': '#a013eb', 'padding': '20px', 'borderRadius': '10px'})
        ])
    ], className="mt-5 mb-5")
], fluid=True)

#############################################
# Interaction Between Components / Controller
#############################################

# Here I added a callback to filter data based on search input or filter type.
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value'), Input('refresh-button', 'n_clicks'), Input('search-input', 'value')]
)
def update_dashboard(filter_type, n_clicks, search_value):
    # Here I added search functionality in the navigation bar to filter by breed or name.
    if filter_type == 'Water Rescue':
        dff = shelter.read({"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}})
    elif filter_type == 'Mountain or Wilderness Rescue':
        dff = shelter.read({"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Border Collie", "Siberian Husky"]}})
    elif filter_type == 'Disaster or Individual Tracking':
        dff = shelter.read({"breed": {"$in": ["Doberman Pinscher", "Bloodhound", "Rottweiler"]}})
    else:
        dff = shelter.read({})

    df_filtered = pd.DataFrame.from_records(dff)

    # Here I dropped the '_id' column from the filtered data if it exists.
    if '_id' in df_filtered.columns:
        df_filtered.drop(columns=['_id'], inplace=True)

    # Here I applied a search filter if a search value is provided to match animal names or breeds.
    # I also made the search case insensitive by converting the search value and relevant fields to lowercase.
    if search_value:
        search_value = search_value.lower()  # Here I verify case insensitivity by converting the search value to lowercase.
        df_filtered = df_filtered[df_filtered.apply(
            lambda row: search_value in row['breed'].lower() or search_value in str(row['name']).lower(), axis=1
        )]

    return df_filtered.to_dict('records')

# Here I added a callback to download data as a CSV file.
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("download-button", "n_clicks"),
    prevent_initial_call=True
)
def download_data(n_clicks):
    # Here I implemented CSV download functionality for ease of data access.
    return dcc.send_data_frame(df.to_csv, "animal_shelter_data.csv")

# Here I added a callback to update the pie chart based on the filtered data.
@app.callback(
    Output('graph-id', "figure"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graph(viewData):
    if viewData:
        dff = pd.DataFrame.from_dict(viewData)
    else:
        dff = df

    # Here I created a pie chart showing breed distribution to visualize the data better.
    fig = px.pie(
        dff,
        names='breed',
        title='Breed Distribution'
    )
    
    return fig

# Here I added a callback to update the map based on the selected data entry from the table.
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', "derived_virtual_selected_rows")],
    [State('datatable-id', 'data')]
)
def update_map(selected_rows, data):
    if not selected_rows or not data:  # Build a default view if there are no selected lines
        markerArray = (30.75, -97.48)  # Default marker at Austin Animal Shelter
        toolTip = "Austin Animal Center"
        popUpHeading = "Austin Animal Center"
        popUpParagraph = "Shelter Home Location"
    else:  # Build the contextual views based on the selection
        dff = pd.DataFrame(data)  # Convert the datatable data to a dataframe
        selected = dff.iloc[selected_rows[0]]  # Get the selected row
        markerArray = (selected['location_lat'], selected['location_long'])  # Use lat/long from selection
        toolTip = selected['breed']  # Ensure correct type for tooltip
        popUpHeading = "Animal Name"
        popUpParagraph = selected['name']  # Ensure correct type for popup paragraph

    return [dl.Map(style={'width': '100%', 'height': '400px'}, center=markerArray,
                   zoom=10, children=[dl.TileLayer(),
                                      dl.Marker(position=markerArray, children=[
                                          dl.Tooltip(toolTip),
                                          dl.Popup([
                                              html.H1(popUpHeading),
                                              html.P(popUpParagraph)
                                          ])
                                      ])
                                     ])
           ]

# Run the Dash app in inline mode
app.run_server(debug=True)

# Print statement to display the URL to the user.
print("Dashboard is running. Go to: http://127.0.0.1:8050")


Dashboard is running. Go to: http://127.0.0.1:8050
